In [1]:
# import findspark
# findspark.init("/u/cs451/packages/spark")

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, Window, functions as F
from pyspark.sql.functions import unix_timestamp, from_unixtime

In [2]:
spark = SparkSession.builder.appName('covid19').getOrCreate()

In [3]:
import random

num_samples = 100000000

def inside(p):     
    x, y = random.random(), random.random()
    return x*x + y*y < 1

In [4]:
spark

In [5]:
days = lambda i: i * 86400

In [6]:
df = spark.read.csv('covid19.csv', header=True)

In [7]:
df.select('*', F.from_unixtime(F.unix_timestamp('date', 'dd-MM-yyyy')).cast('timestamp').alias('date_'))\
    .orderBy('date_')\
    .toPandas()

,pruid,prname,prnameFR,date,numconf,numprob,numdeaths,numtotal,numtested,numrecover,percentrecover,ratetested,numtoday,percentoday,date_
0,35,Ontario,Ontario,31-01-2020,3,0,0,3,None,N/A,None,None,3,3.000,2020-01-31
1,59,British Columbia,Colombie-Britannique,31-01-2020,1,0,0,1,None,N/A,None,None,1,1.000,2020-01-31
2,1,Canada,Canada,31-01-2020,4,0,0,4,None,N/A,None,None,4,4.000,2020-01-31
3,35,Ontario,Ontario,08-02-2020,3,0,0,3,None,N/A,None,None,0,0.000,2020-02-08
4,59,British Columbia,Colombie-Britannique,08-02-2020,4,0,0,4,None,N/A,None,None,3,3.000,2020-02-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,60,Yukon,Yukon,13-04-2020,8,0,0,8,832,None,None,None,None,None,2020-04-13
563,61,Northwest Territories,Territoires du Nord-Ouest,13-04-2020,5,0,0,5,1390,None,None,None,None,None,2020-04-13
564,62,Nunavut,Nunavut,13-04-2020,0,0,0,0,264,None,None,None,None,None,2020-04-13
565,99,Repatriated travellers,Voyageurs rapatriés,13-04-2020,13,0,None,13,None,None,None,None,None,None,2020-04-13


In [8]:
df.filter(df.prname == 'Canada')\
    .select(F.from_unixtime(F.unix_timestamp('date', 'dd-MM-yyyy')).cast('timestamp').alias('date'),
            'numconf',
            'numprob',
            'numdeaths',
            'numtotal',
            'numtested',
            'numrecover',
            'numtoday'
         )\
    .withColumn('rollingaverage', 
                F.avg("numconf").over(Window.orderBy(F.col("date").cast('long')).rangeBetween(-days(6), days(0))))\
    .withColumn('numconf_lag_7', 
                F.first("numconf").over(Window.orderBy(F.col("date").cast('long')).rangeBetween(-days(7), -days(7))))\
    .withColumn('numconf_lag_6', 
                F.first("numconf").over(Window.orderBy(F.col("date").cast('long')).rangeBetween(-days(6), -days(6))))\
    .withColumn('numconf_lag_5', 
                F.first("numconf").over(Window.orderBy(F.col("date").cast('long')).rangeBetween(-days(5), -days(5))))\
    .withColumn('numconf_lag_4', 
                F.first("numconf").over(Window.orderBy(F.col("date").cast('long')).rangeBetween(-days(4), -days(4))))\
    .withColumn('numconf_lag_3', 
                F.first("numconf").over(Window.orderBy(F.col("date").cast('long')).rangeBetween(-days(3), -days(3))))\
    .withColumn('numconf_lag_2', 
                F.first("numconf").over(Window.orderBy(F.col("date").cast('long')).rangeBetween(-days(2), -days(2))))\
    .withColumn('numconf_lag_1', 
                F.first("numconf").over(Window.orderBy(F.col("date").cast('long')).rangeBetween(-days(1), -days(1))))\
    .toPandas()

,date,numconf,numprob,numdeaths,numtotal,numtested,numrecover,numtoday,rollingaverage,numconf_lag_7,numconf_lag_6,numconf_lag_5,numconf_lag_4,numconf_lag_3,numconf_lag_2,numconf_lag_1
0,2020-01-31,4,0,0,4,None,N/A,4,4.000000,None,None,None,None,None,None,None
1,2020-02-08,7,0,0,7,None,N/A,3,7.000000,None,None,None,None,None,None,None
2,2020-02-16,8,0,0,8,None,N/A,1,8.000000,None,None,None,None,None,None,None
3,2020-02-21,9,0,0,9,None,N/A,1,8.500000,None,None,8,None,None,None,None
4,2020-02-24,10,0,0,10,None,N/A,1,9.500000,None,None,None,None,9,None,None
5,2020-02-25,11,0,0,11,None,N/A,1,10.000000,None,None,None,9,None,None,10
6,2020-02-26,12,0,0,12,None,N/A,1,10.500000,None,None,9,None,None,10,11
7,2020-02-27,13,0,0,13,None,N/A,1,11.000000,None,9,None,None,10,11,12
8,2020-02-29,15,0,0,15,None,N/A,2,12.200000,None,None,10,11,12,13,None
9,2020-03-01,24,0,0,24,None,N/A,9,14.166667,None,10,11,12,13,None,15


In [90]:
(17883+19274+20748+22133+23301+24365+25663)/7

21909.571428571428

In [91]:
(24365 + 25663)/2

25014.0

In [132]:
23301 - 22133

1168